# Dash Application (CNN Plant Identification)

In [1]:
import pandas as pd
import numpy as np

import cv2
import base64

import os
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
import contextlib

from keras.models import Sequential, Model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

import tensorflow as tf
import keras

from PIL import Image
import urllib

import sklearn
from sklearn import preprocessing

import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State

import plotly.express as px
import plotly.graph_objects as go

## Import Data + Keras Model
### Insert the paths of the files generated in DF Assembly.

In [17]:
info_df = pd.read_csv(r'C:\Users\David\Documents\code\What_Plant\info_df')
df = pd.read_pickle(r'C:\Users\David\Documents\code\What_Plant\cucumber')
model = keras.models.load_model(r'C:\Users\David\Documents\code\Final_Project')

## Prepare labels

In [9]:
#Label Prep
le = preprocessing.LabelEncoder() #We will need this to return our original labels later
y = df.label
le.fit(y)
y = le.transform(y)
output_shape = len(np.unique(y))

y = to_categorical(y)

## Logo, for style
### This really isn't necessary. Consider replacing the dcc.Graph in the top row with a regular title/

In [6]:
img = cv2.imread(r'C:\Users\David\Documents\code\What_Plant\Banner.jpg')
pic = px.imshow(img)
pic.update_layout(coloraxis_showscale=False,
        margin_l=0,
        margin_r=0,
        margin_t=0,
        margin_b=0)
pic.update_xaxes(showticklabels=False)
pic.update_yaxes(showticklabels=False);

## Construct Dash App and Deploy

In [18]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MINTY],
                meta_tags=[{'name': 'viewport', 'content': 'width=device-width, initial-scale=1.0'}]
                )
server = app.server
# ----------------------------------------------------------------------------------------------------
app.layout = dbc.Container([

# First Row
dbc.Row([
        dbc.Col(dcc.Graph(figure=pic))
                
    ]),
#Second Row
    dbc.Row([
        dcc.Upload(
        id='upload-image',
        children=html.Div([
            'Drop your Plant Picture or Click to ',
            html.A('Select a File')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        multiple=False
    ),

    ]),

    dbc.Row([
        dbc.Col(html.Img(id='output-image-upload', height=600, width=600),
                width=7),
        dbc.Col(dcc.Graph(id='Info_Chart', className="thead-dark"),
                width=5)
    ]),


])

def make_chart(plant):
    df = info_df.loc[info_df.latin == plant]
    fig = go.Figure(
        data=[go.Table(header=dict(values=['Light', 'Soil', 'Water']),
                       cells=dict(values=[df['Light'], df['Soil'], df['Water'] ]) )
              ])
    fig.update_layout(
        title=dict(
            text=(plant + ' Care Information'),
            x=0.5
        ),
        font_family='arial',
        margin_l=0,
        margin_r=0,
        margin_t=30,
        margin_b=0)
    return fig



# ---------------------------------------------------------------
# Info_Chart
@app.callback(
    Output('Info_Chart', 'figure'),
    Output('output-image-upload', 'src'),
    Input('upload-image', 'contents'))
def update_all(contents):
    if contents is None:
        pred = 'Lithops'
        fig = make_chart(pred)
        src = df.loc[df.label == pred, 'image_url'][:1]
        return fig, src

        raise dash.exceptions.PreventUpdate

    #Return prediction
    encoded_data = contents.split(',')[1]
    nparr = np.frombuffer(base64.b64decode(encoded_data), np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    image = cv2.resize(img, (28, 28), 0, 0, cv2.INTER_LINEAR)
    X = image.reshape(-1, 28, 28, 3)
    predictions = np.argmax(model.predict(X), axis=-1) 
    pred = le.inverse_transform(predictions)
    pred = pred[0]

    #Return InfoChart
    fig = make_chart(pred)

    #Return Image
    src = df.loc[df.label == pred, 'image_url'][:1]
    return fig, src

# ---------------------------------------------------------------
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/May/2021 17:54:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2021 17:54:29] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2021 17:54:29] "GET /_favicon.ico?v=1.20.0 HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2021 17:54:29] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2021 17:54:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2021 17:54:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2021 17:54:43] "POST /_dash-update-component HTTP/1.1" 200 -
